# Private RAG Application Implementation Using Llama 3.2, Ollama & PostgreSQL

In [1]:
import psycopg2

## Dataset Preparation

In [2]:
# the knowledge base
dummy_data = [
    {"title": "Seoul Tower", "content": "Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea."},
    {"title": "Gwanghwamun Gate", "content": "Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea."},
    {"title": "Bukchon Hanok Village", "content": "Bukchon Hanok Village is a Korean traditional village in Seoul with a long history."},
    {"title": "Myeong-dong Shopping Street", "content": "Myeong-dong is one of the primary shopping districts in Seoul, South Korea."},
    {"title": "Dongdaemun Design Plaza", "content": "The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea."}
]

## Connect to the database

In [3]:
def connect_db():
    return psycopg2.connect( # use the credentials of your postgresql database 
        host = 'localhost',
        database = 'postgres',
        user = 'postgres',
        password = 'password',
        port = '5432'
    )

## Create the table

In [4]:
conn = connect_db()
cur = conn.cursor()
cur.execute("""
        CREATE TABLE IF NOT EXISTS documents (
            id SERIAL PRIMARY KEY,
            title TEXT,
            content TEXT,
            embedding VECTOR(768)
        );
    """)
conn.commit()
cur.close()
conn.close()

### Insert the data and embeddings into the database

In [5]:
conn = connect_db()
cur = conn.cursor()

# use the port at which your ollama service is running.
for doc in dummy_data:
    cur.execute("""
        INSERT INTO documents (title, content, embedding)
        VALUES (
            %(title)s,
            %(content)s,
            ollama_embed('nomic-embed-text', concat(%(title)s, ' - ', %(content)s), _host=>'http://ollama:11434')
        )
    """, doc)

conn.commit()
cur.close()
conn.close()

Safechecking the insert information in the database.

In [6]:
conn = connect_db()
cur = conn.cursor()
    
cur.execute("""
    SELECT title, content, vector_dims(embedding) 
    FROM documents;
""")

rows = cur.fetchall()
for row in rows:
    print(f"Title: {row[0]}, Content: {row[1]}, Embedding Dimensions: {row[2]}")

cur.close()
conn.close()

Title: Seoul Tower, Content: Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea., Embedding Dimensions: 768
Title: Gwanghwamun Gate, Content: Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea., Embedding Dimensions: 768
Title: Bukchon Hanok Village, Content: Bukchon Hanok Village is a Korean traditional village in Seoul with a long history., Embedding Dimensions: 768
Title: Myeong-dong Shopping Street, Content: Myeong-dong is one of the primary shopping districts in Seoul, South Korea., Embedding Dimensions: 768
Title: Dongdaemun Design Plaza, Content: The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea., Embedding Dimensions: 768


## Retrieval and Generation

### Define the query and its embedding

In [7]:
query = "Tell me about gates in South Korea."

In [8]:
conn = connect_db()
cur = conn.cursor()
    
# Embed the query using the ollama_embed function
cur.execute("""
    SELECT ollama_embed('nomic-embed-text', %s, _host=>'http://ollama:11434');
""", (query,))
query_embedding = cur.fetchone()[0]

# Retrieve relevant documents based on cosine distance
cur.execute("""
    SELECT title, content, 1 - (embedding <=> %s) AS similarity
    FROM documents
    ORDER BY similarity DESC
    LIMIT 3;
""", (query_embedding,))

rows = cur.fetchall()
    
# Prepare the context for generating the response
context = "\n\n".join([f"Title: {row[0]}\nContent: {row[1]}" for row in rows])
print(context)

cur.close()
conn.close()

Title: Gwanghwamun Gate
Content: Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea.

Title: Dongdaemun Design Plaza
Content: The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea.

Title: Seoul Tower
Content: Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea.


In [9]:
conn = connect_db()
cur = conn.cursor()

# Generate the response using the ollama_generate function
cur.execute("""
    SELECT ollama_generate('llama3.2', %s, _host=>'http://ollama:11434');
""", (f"Query: {query}\nContext: {context}",))
    
model_response = cur.fetchone()[0]
print(model_response['response'])
    
cur.close()
conn.close()

Gates in South Korea play a significant role in the country's rich history, architecture, and culture. Here are some key aspects of gates in South Korea:

1. Historical significance: Many gates in South Korea have historical significance, serving as entrance points to palaces, fortresses, and other important sites. The Gwanghwamun Gate, mentioned earlier, is a notable example.
2. Architectural style: Traditional Korean gates often feature unique architectural styles, such as the use of wooden materials, intricate carvings, and ornate decorations. These gates can provide insight into the country's rich cultural heritage.
3. National symbols: Some gates in South Korea have become national symbols or landmarks, attracting tourists and locals alike. The Gwanghwamun Gate is one example, while others include the gate at Bulguksa Temple, a UNESCO World Heritage Site.
4. Urban development: Modern gates in South Korea often serve as part of urban development projects, such as the Dongdaemun Des